<a href="https://colab.research.google.com/github/yukinaga/automl/blob/main/section_2/01_automl_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoMLによる「回帰」
ライブラリPyCaretを使い、AutoMLによる「回帰」の機械学習を実装します。  
ダイヤモンドの様々な観測データから、その価格を予測します。  

## PyCaretのインストール
AutoMLをサポートするライブラリ、PyCaretをバージョンを指定してインストールします。

In [ ]:
!pip install pycaret==2.3.6
!pip install pandas-profiling==3.1.0

## Google Colabの設定
Google Colab環境でPyCaretのインタラクティブな要素を表示するためには、以下のコードを実行する必要があります。

In [ ]:
from pycaret.utils import enable_colab

enable_colab()

## データセットの読み込み
今回は「Sarah Gets a Diamond」というデータセットを使います。  
データセットには6000レコードが含まれています。  
各カラムの簡単な説明は以下の通りです。

* ID: 各ダイヤを識別するためのID
* Carat Weight: ダイヤの重量
* Cut: ダイヤモンドのカットを望ましい順に並べたもの（Signature-Ideal、Ideal、Very Good、Good、Fair)
* Color: ダイヤモンドの色（D、E、Fは無色、G、H、Iは無色に近い）
* Clarity: ダイヤモンドのクラリティ（F - Flawless、IF - Internally Flawless、VVS1またはVVS2 - Very, Very Slightly Included、VS1またはVS2 - Very Slightly Included、SI1 - Slightly Included）
* Polish: ダイヤモンドの研磨状態（ID - Ideal、EX - Excellent、VG - Very Good、G - Good）
* Symmetry: ダイヤモンドの対称性（ID - 理想的、EX - 優れている、VG - 非常に良い、G - 良い）
* Report: 「 AGSL」または「GIA」のいずれかで、ダイヤの品質を報告した鑑定機関を示す
* Price: ダイヤモンドの評価額（米ドル）

https://www.kaggle.com/c/abafall2020/data

In [ ]:
from pycaret.datasets import get_data

dataset = get_data("diamond")  # データの取得
dataset.shape  # データの形状

訓練済みモデルの検証に用いる「未知のデータ」を訓練データから取り出します。  

In [ ]:
data = dataset.sample(frac=0.9, random_state=786)  # 訓練データ
data_unseen = dataset.drop(data.index)  # 未知のデータ

data.reset_index(drop=True, inplace=True)  # インデックスを初期化
data_unseen.reset_index(drop=True, inplace=True)  # インデックスを初期化

print("訓練データの形状: " + str(data.shape))
print("未知のデータの形状: " + str(data_unseen.shape))

## 環境の設定
PyCaretの環境を設定します。  
setup関数はPyCaretの環境を初期化しますが、PyCaretの他の関数を実行する前に呼び出す必要があります。  

以下のコードを実行すると、setup関数が実行されてすべての特徴のデータ型が自動的に推定されます。  
この際に、必ずしも正しく推論されるとは限らないことにご注意ください。     
データの型に問題が無ければ、空白を入力することで設定を完了することができます。  

In [ ]:
from pycaret.regression import setup

clf = setup(data=data, target="Price", session_id=123)  # 環境の初期化

## モデルの比較
様々なモデルを比較して、性能を評価します。  
compare_models関数は、ライブラリ内のすべてのモデルを使って訓練を行い、スコアを評価します。  

In [ ]:
from pycaret.regression import compare_models

best_model = compare_models()  # 全てのモデルを訓練し、評価する

わずか1行のコードで、15以上の機械学習モデルを訓練し、評価することができました。  
  
最もスコアの良いモデルの概要を表示します。


In [ ]:
print(best_model)

## モデルの作成
models関数により、全ての使用可能な機械学習モデルを確認することができます。

In [ ]:
from pycaret.regression import models

models()  # 機械学習モデルの一覧

create_model関数は、「交差検証」を用いて個別のモデルの訓練と評価を行います。  
デフォルトでは、データを10に分割し10通りのテストデータ/訓練データの組み合わせを試します。     
  
今回は、上記の中で最もシンプルな「線形回帰」のモデルを作成します。  

In [ ]:
from pycaret.regression import create_model

lr = create_model("lr")  # 線形回帰のモデルを作成

訓練済みモデルの概要を表示します。  

In [ ]:
print(lr)

各ハイパーパラメータの意味は、scikit-learnの公式ドキュメントに解説があります。  
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

## ハイパーパラメータの調整
create_model関数は、デフォルトのハイパーパラメータを使ってモデルを訓練します。  
ハイパーパラメータを調整するためには、tune_model関数を使用します。  

In [ ]:
from pycaret.regression import tune_model

tuned_lr = tune_model(lr)  # ハイパーパラメータの調整

ハイパーパラメータを調整済みの、訓練済みモデルの概要を表示します。  

In [ ]:
print(tuned_lr)

## モデルを評価する
plot_model関数により、様々な角度からモデルを分析することができます。  
https://pycaret.org/plot-model/

plot_model関数でplotを指定しない場合、残差プロット（Residual Plot）が表示されます。  
残差プロットは横軸に予測値をとり、縦軸に回帰残差をとってプロットしたものです。  
これにより、何らかの規則性や分散の偏りなどの有無を判断することができます。 

In [ ]:
from pycaret.regression import plot_model

plot_model(tuned_lr)

以下のコードは、各特徴量の重要度をプロットします。

In [ ]:
plot_model(tuned_lr, plot="feature")  # 各特徴の重要度をプロット

モデルは、`evaluate_model`関数で評価することもできます。  

In [ ]:
from pycaret.regression import evaluate_model

evaluate_model(tuned_lr)

## デプロイ用のモデルを作成
`finalize_model`関数により全てのデータセットを使ってモデルを訓練し、デプロイ用のモデルを作成します。   

In [ ]:
from pycaret.regression import finalize_model

final_lr = finalize_model(tuned_lr)
print(final_lr)

`predict_model`関数によりデプロイ用のモデルを評価します。

In [ ]:
from pycaret.regression import predict_model

predict_model(final_lr);

`save_model`関数によりモデルを保存することができます。  

In [ ]:
from pycaret.regression import save_model

save_model(final_lr, "lr_model")

`load_model`関数により、保存されたモデルを読み込むことができます。

In [ ]:
from pycaret.regression import load_model

loaded_lr = load_model("lr_model")

## 未知のデータで予測する
デプロイ用のモデルを使い、「未知のデータ」で価格の予測を行います。  

In [ ]:
unseen_pred = predict_model(loaded_lr, data=data_unseen)
unseen_pred.head()